## Packages Import

In [1]:
from flask import Flask
from datetime import datetime
from flask import render_template
from flask import request
from flask import url_for
from flask import redirect

import requests
import json
import os
import pandas as pd

## Flask app 선언

In [2]:
app = Flask(__name__, template_folder=os.getcwd())

## login 기능
#### 첫 접속시에는 login.html 파일을 랜더링
#### POST 방식으로 지갑 주소를 입력한 경우,
#### > 알맞은 지갑주소일 경우 wallet.html 파일 랜더링
#### > 잘못된 지갑주소일 경우 에러 메시지 띄움

In [3]:
@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method=='POST':
        print("login 버튼을 누름")
        input_value = request.form.to_dict(flat=False) ['wallet_id'][0]
        print("login 지갑주소 : " , input_value)
        ### 기존 user 정보 확인
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        res = requests.get("http://localhost:5000/chain", headers=headers)
        status_json = json.loads(res.text)
        status_json['chain']    
        tx_amount_l = []
        tx_sender_l = []
        tx_reciv_l  = []
        tx_time_l   = []
        # 거래내역 정리 (df_tx)
        for chain_index in range(len(status_json['chain'])):
            chain_tx = status_json['chain'][chain_index]['transactions']
            for each_tx in range(len(chain_tx)):
                tx_amount_l.append(chain_tx[each_tx]['amount'])
                tx_sender_l.append(chain_tx[each_tx]['sender'])
                tx_reciv_l.append(chain_tx[each_tx]['recipient'])
                tx_time_l.append(chain_tx[each_tx]['timestamp'])
        df_tx = pd.DataFrame()
        df_tx['timestamp'] = tx_time_l  
        df_tx['sender'] = tx_sender_l 
        df_tx['recipient'] = tx_reciv_l
        df_tx['amount'] = tx_amount_l   
        df_tx

        # pyBTC 잔고현황 정리 (df_status)
        df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
        df_sended.columns = ['user','sended_amount']
        df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
        df_received.columns = ['user','received_amount']
        df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
        df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
        df_status       
    
        # 결과값 랜더링
        if (df_status['user']==input_value ).sum() == 1:
            print("로그인성공")
            return render_template("wallet.html",  wallet_id = input_value, 
                                                    wallet_value = df_status[df_status['user']== input_value]['balance'].iloc[0])
        else:
            return "잘못된 지갑주소입니다."
        
    return render_template('login.html')


## 지갑 기능
#### POST 방식으로 보내는 지갑 주소와 보내는 pyBTC 양이 입력된 경우,
#### 해당 지갑주소일 pyBTC 전송

In [4]:
@app.route('/wallet', methods=['GET', 'POST'])
def wallet():
    if request.method=='POST':
        send_value = int(request.form.to_dict(flat=False)['send_value'][0] )
        send_target = request.form.to_dict(flat=False)['send_target'][0]
        send_from = request.form.to_dict(flat=False)['send_from'][0]
        print("Login Wallet ID : " ,send_from)
        
        if send_value > 0:
            print("Send Amout :", send_value)
            ## transaction 입력하기
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "sender": send_from,
                "recipient": send_target,
                "amount": send_value,
            }
            requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data))

            return "전송 완료!"

        else:
            return "0 pyBTC 이상 보내주세요!"
        
    return render_template('wallet.html')

## 지갑 사이트 실행

In [5]:
app.run(port=8081)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8081/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Dec/2022 18:25:09] "GET / HTTP/1.1" 200 -


login 버튼을 누름
login 지갑주소 :  test_to


127.0.0.1 - - [04/Dec/2022 18:25:16] "POST / HTTP/1.1" 200 -


로그인성공
Login Wallet ID :  test_to
Send Amout : 3


...

로그인성공


127.0.0.1 - - [04/Dec/2022 18:33:08] "GET / HTTP/1.1" 200 -


login 버튼을 누름
login 지갑주소 :  test_to


127.0.0.1 - - [04/Dec/2022 18:33:16] "POST / HTTP/1.1" 200 -


로그인성공


### [참고] df_status 에 저장되는 내역

In [6]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

status_json = json.loads(res.text)
status_json['chain']    
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])
df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l  
df_tx['sender'] = tx_sender_l 
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l   
df_tx

df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status     

,user,received_amount,sended_amount,balance
0,node_5000,0.2,0.0,0.2
1,test_to,3.0,0.0,3.0
2,test_to2,30.0,0.0,30.0
3,test_to3,300.0,0.0,300.0
4,master,0.0,0.2,-0.2
5,test_from,0.0,333.0,-333.0


In [13]:
df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status

,user,received_amount,sended_amount,balance
0,24a905ff16514caab92441fed3dd69e7e641e0124110b7...,18.0,18.0,0.0
1,2f3532702ecc166ea0b4b1c129d3f913c1c69b0987c818...,40.0,0.0,40.0
2,9cb89af232da391ced43390624e489ba105458e0cb487f...,96.0,0.0,96.0
3,cb98707469fbc97f35777b89eb69a7491ac46a26b0e43d...,74.0,0.0,74.0
4,d80fa0b7f70d0b5abfc4f70353ec944c96021406001fd6...,110.0,0.0,110.0
5,e2fa281fb59fae141752752c640d278af56fa1677a00a0,18.0,0.0,18.0
6,e2fa281fb59fae141752752c640d278af56fa1677a00a0...,195.0,0.0,195.0
7,node_5000,0.4,0.0,0.4
8,1aa1640f66c9e2e901751a1c33c99246912fc3ffbfe809...,0.0,96.0,-96.0
9,2d9efc41b538e9f418ecf119b1a71c29beea87d9fbde99...,0.0,40.0,-40.0


In [2]:
    headers = {'Content-Type' : 'application/json; charset=utf-8'}
    res = requests.get("http://localhost:5000/chain", headers=headers)
    print(res.text )
    print("*"*8)
    status_json = json.loads(res.text)
    status_json['chain']    
    tx_amount_l = []
    tx_sender_l = []
    tx_reciv_l  = []
    tx_time_l   = []

    for chain_index in range(len(status_json['chain'])):
        chain_tx = status_json['chain'][chain_index]['transactions']
        for each_tx in range(len(chain_tx)):
            tx_amount_l.append(chain_tx[each_tx]['amount'])
            tx_sender_l.append(chain_tx[each_tx]['sender'])
            tx_reciv_l.append(chain_tx[each_tx]['recipient'])
            tx_time_l.append(chain_tx[each_tx]['timestamp'])

    df_tx = pd.DataFrame()
    df_tx['timestamp'] = tx_time_l  
    df_tx['sender'] = tx_sender_l 
    df_tx['recipient'] = tx_reciv_l
    df_tx['amount'] = tx_amount_l   
    df_tx



    df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
    df_sended.columns = ['user','sended_amount']
    df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
    df_received.columns = ['user','received_amount']
    df_received


{"chain":[{"index":1,"nonce":100,"previous_hash":1,"timestamp":1670003137.3593862,"transactions":[]},{"index":2,"nonce":889040,"previous_hash":"f2a3e59909517450db4a8d714c3382a2d3e6911122d03d83168e7d1d6ab39d62","timestamp":1670003156.9233766,"transactions":[{"amount":74,"recipient":"cb98707469fbc97f35777b89eb69a7491ac46a26b0e43d2add7571157c245078","sender":"a28d2d0562eabe427e0572a40c0ff19784e7ec7b889eca864be130b64fd85149","timestamp":1670003154.7843897},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1670003156.9223733}]},{"index":3,"nonce":740763,"previous_hash":"24ef618532580c6789d082d9ed72f749ed92be471c1996695611fcbfedfca508","timestamp":1670003167.1263945,"transactions":[{"amount":110,"recipient":"d80fa0b7f70d0b5abfc4f70353ec944c96021406001fd68f338d2e1d3a1a0983","sender":"ff36a2f62b1ca79ed974d48f1a2bc00fe49c680754f6acaebab35164098f6d20","timestamp":1670003158.9508111},{"amount":40,"recipient":"2f3532702ecc166ea0b4b1c129d3f913c1c69b0987c818fc5e5f0f440f554c8b","sen

,user,received_amount
0,24a905ff16514caab92441fed3dd69e7e641e0124110b7...,18.0
1,2f3532702ecc166ea0b4b1c129d3f913c1c69b0987c818...,40.0
2,9cb89af232da391ced43390624e489ba105458e0cb487f...,96.0
3,cb98707469fbc97f35777b89eb69a7491ac46a26b0e43d...,74.0
4,d80fa0b7f70d0b5abfc4f70353ec944c96021406001fd6...,110.0
5,e2fa281fb59fae141752752c640d278af56fa1677a00a0...,195.0
6,node_5000,0.3


In [14]:
    headers = {'Content-Type' : 'application/json; charset=utf-8'}
    res = requests.get("http://localhost:5000/chain", headers=headers)
    print(res.text )
    print("*"*8)
    status_json = json.loads(res.text)
    status_json['chain']    
    tx_amount_l = []
    tx_sender_l = []
    tx_reciv_l  = []
    tx_time_l   = []

    for chain_index in range(len(status_json['chain'])):
        chain_tx = status_json['chain'][chain_index]['transactions']
        for each_tx in range(len(chain_tx)):
            tx_amount_l.append(chain_tx[each_tx]['amount'])
            tx_sender_l.append(chain_tx[each_tx]['sender'])
            tx_reciv_l.append(chain_tx[each_tx]['recipient'])
            tx_time_l.append(chain_tx[each_tx]['timestamp'])

    df_tx = pd.DataFrame()
    df_tx['timestamp'] = tx_time_l  
    df_tx['sender'] = tx_sender_l 
    df_tx['recipient'] = tx_reciv_l
    df_tx['amount'] = tx_amount_l   
    df_tx



    df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
    df_sended.columns = ['user','sended_amount']
    df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
    df_received.columns = ['user','received_amount']
    df_received
    
    df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
    df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
    df_status


{"chain":[{"index":1,"nonce":100,"previous_hash":1,"timestamp":1670003137.3593862,"transactions":[]},{"index":2,"nonce":889040,"previous_hash":"f2a3e59909517450db4a8d714c3382a2d3e6911122d03d83168e7d1d6ab39d62","timestamp":1670003156.9233766,"transactions":[{"amount":74,"recipient":"cb98707469fbc97f35777b89eb69a7491ac46a26b0e43d2add7571157c245078","sender":"a28d2d0562eabe427e0572a40c0ff19784e7ec7b889eca864be130b64fd85149","timestamp":1670003154.7843897},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1670003156.9223733}]},{"index":3,"nonce":740763,"previous_hash":"24ef618532580c6789d082d9ed72f749ed92be471c1996695611fcbfedfca508","timestamp":1670003167.1263945,"transactions":[{"amount":110,"recipient":"d80fa0b7f70d0b5abfc4f70353ec944c96021406001fd68f338d2e1d3a1a0983","sender":"ff36a2f62b1ca79ed974d48f1a2bc00fe49c680754f6acaebab35164098f6d20","timestamp":1670003158.9508111},{"amount":40,"recipient":"2f3532702ecc166ea0b4b1c129d3f913c1c69b0987c818fc5e5f0f440f554c8b","sen

,user,received_amount,sended_amount,balance
0,24a905ff16514caab92441fed3dd69e7e641e0124110b7...,18.0,18.0,0.0
1,2f3532702ecc166ea0b4b1c129d3f913c1c69b0987c818...,40.0,0.0,40.0
2,9cb89af232da391ced43390624e489ba105458e0cb487f...,96.0,0.0,96.0
3,cb98707469fbc97f35777b89eb69a7491ac46a26b0e43d...,74.0,0.0,74.0
4,d80fa0b7f70d0b5abfc4f70353ec944c96021406001fd6...,110.0,0.0,110.0
5,e2fa281fb59fae141752752c640d278af56fa1677a00a0,18.0,0.0,18.0
6,e2fa281fb59fae141752752c640d278af56fa1677a00a0...,195.0,0.0,195.0
7,node_5000,0.4,0.0,0.4
8,1aa1640f66c9e2e901751a1c33c99246912fc3ffbfe809...,0.0,96.0,-96.0
9,2d9efc41b538e9f418ecf119b1a71c29beea87d9fbde99...,0.0,40.0,-40.0


In [1]:
from flask import Flask
from datetime import datetime
from flask import render_template
from flask import request
from flask import url_for
from flask import redirect

import requests
import json
import os
import pandas as pd


app = Flask(__name__, template_folder=os.getcwd())

@app.route('/', methods=['GET', 'POST'])
def login():

    if request.method=='POST':
        print("login 버튼을 누름")
        input_value = request.form.to_dict(flat=False)
        print("login 지갑주소 : " , input_value)
        
        ### 기존 user 정보 확인
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        res = requests.get("http://localhost:5000/chain", headers=headers)
#         print(res.text )
        print("*"*8)
        status_json = json.loads(res.text)
        status_json['chain']    
        tx_amount_l = []
        tx_sender_l = []
        tx_reciv_l  = []
        tx_time_l   = []

        for chain_index in range(len(status_json['chain'])):
            chain_tx = status_json['chain'][chain_index]['transactions']
            for each_tx in range(len(chain_tx)):
                tx_amount_l.append(chain_tx[each_tx]['amount'])
                tx_sender_l.append(chain_tx[each_tx]['sender'])
                tx_reciv_l.append(chain_tx[each_tx]['recipient'])
                tx_time_l.append(chain_tx[each_tx]['timestamp'])

        df_tx = pd.DataFrame()
        df_tx['timestamp'] = tx_time_l  
        df_tx['sender'] = tx_sender_l 
        df_tx['recipient'] = tx_reciv_l
        df_tx['amount'] = tx_amount_l   
        df_tx



        df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
        df_sended.columns = ['user','sended_amount']
        df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
        df_received.columns = ['user','received_amount']
        df_received

        df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
        df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
        df_status       
    
    
        if (df_status['user']==input_value['wallet_id'][0] ).sum() == 1:
            print("로그인성공")
            return render_template("wallet.html",  wallet_id = input_value['wallet_id'][0], 
                                                    wallet_value = df_status[df_status['user']== df_status['user'].iloc[0]]['balance'].iloc[0])
        else:
            return "잘못된 지갑주소입니다."
        
    return render_template('login.html')

@app.route('/wallet', methods=['GET', 'POST'])
def wallet():
    if request.method=='POST':
        send_value = int(request.form.to_dict(flat=False)['send_value'][0] )
        send_target = request.form.to_dict(flat=False)['send_target'][0]
        send_from = request.form.to_dict(flat=False)['send_from'][0]
        print("123123 : " ,send_from)
        
        if send_value > 0:
            print(send_value)
            ## transaction 입력하기
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "sender": send_from,
                "recipient": send_target,
                "amount": send_value,
            }
            requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

            return "전송 완료!"

        else:
            return "0 pyBTC 이상 보내주세요!"

        
        
    return render_template('wallet.html')
    
    
    
app.run(port=8081)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8081/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Dec/2022 17:15:20] "GET / HTTP/1.1" 200 -


In [12]:
        if (df_received['user']==input_value['wallet_id'][0] ).sum() == 1:
            print("로그인성공")
            return "로그인성공!!!!!!"

1

In [6]:
 df_received['user'].iloc[0]

'0159188535cfea794788f455dcb88969aed597336b6168ae39c72cc615f77420'

In [15]:
df_received[df_received['user']== df_received['user'].iloc[0]]['received_amount'].iloc[0]

40.0

In [ ]:
788490d516fac98d48a2c7bd6ad73e0c183de30a1ccf048c5cc50345aeaaa03b